创建一个文件夹openai-agent
`cd openai-agent`
加入该文件夹/在该文件夹打开终端

In [ ]:
设置虚拟环境openai-agent
python -m venv openai-agent

# 如果是mac或者linux系统，激活虚拟环境：
source openai-agent/bin/activate

In [ ]:
# 确保pip是最新版本
python3 -m pip install --upgrade pip
# 查看当前所有pip
python3 -m pip list
# 查看某个pip版本
python3 -m pip show <package-name>


In [ ]:
# 安装依赖
python3 -m pip install "openai-agents[litellm]"
pip install python-dotenv # 安装dotenv库，管理环境变量
# 配置环境变量
# 目录下新增一个`.env`文件，内容如下：
DEEPSEEK_API_KEY="sk-xxxx..."
# 配置完成后，重启终端

注册jupyter内核
虚拟环境中激活环境
```bash
cd /Users/archroot/Documents/Obsidian/wow-agent/agent-learning
source bin/activate
```
安装ipykernel包：
```bash
pip install ipykernel
```
设置虚拟环境为Jupyter内核
```bash
python -m ipykernel install --user --name=agent-learning --display-name="Python (agent-learning)"
```

创建python文件，以下步骤也可以分两步在jupyter运行

In [ ]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = "https://api.deepseek.com/v1"
chat_model = "deepseek/deepseek-chat"

In [ ]:
from litellm import completion

response = completion(
    model="ollama/qwen2.5:7b", 
    messages=[{ "content": "你有什么技能？","role": "user"}], 
    api_base="http://192.168.0.123:11434"
)
print(response.choices[0].message.content)

以上的`LitellmModel`，它像一个“万能转换插头”，非常强大。但教程接下来介绍的这种方法，是直接利用`openai`官方库的客户端`AsyncOpenAI`来配置。

其方法与`LitellmModel`的配置方法相同，只是将`openai`替换为`AsyncOpenAI`。其中的`AsyncOpenAI`客户端是异步的，而`openai`客户端是同步的。
`openai-agents`框架的核心逻辑与具体是哪个模型解耦了。它既可以接受像`LitellmModel`这样的通用适配器，也可以接受一个配置好的、符合规范的客户端。

以下代码为异步运行的示例，不建议直接在jupyter notebook中运行，因为异步代码需要在事件循环中运行。
`result = Runner.run_sync(agent, "给我讲个程序员相亲的笑话")`这个方法是同步运行的，会阻塞当前线程，直到模型返回结果。
可以通过`import asyncio`引入异步io库，然后使用`asyncio.run()`方法运行异步代码。
```python
# 定义异步函数，用于执行异步逻辑
async def main():
    # 使用异步运行方法（通常框架会提供如 Runner.run 的异步接口）
    result = await Runner.run(agent, "给我讲个程序员相亲的笑话")
    print(result.final_output)
```

In [ ]:
from agents import Agent, Runner, set_tracing_disabled, set_default_openai_api, set_default_openai_client
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

chat_model = "glm-4-flash"
client = AsyncOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4/",
    api_key=os.getenv('zhipu_key'),
)
set_default_openai_client(client=client, use_for_tracing=False) # 全局变量
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

agent = Agent(name="Assistant", model=chat_model, instructions="You are a helpful assistant")
result = Runner.run_sync(agent, "给我讲个程序员相亲的笑话")
print(result.final_output)

一种是通用的`LitellmModel`，另一种是针对OpenAI兼容模型的`set_default_openai_client`全局设置。
下方示例，使用了引入了一个更高级、更灵活的模型管理概念：`ModelProvider`（模型提供者）。这是`openai-agents`框架中一个非常优雅的设计，彻底实现了代码的解耦。
- Agent 类负责定义智能体的“身份”和“职责”（name, instructions）。
- ModelProvider 类负责“生产”具体的模型实例（get_model）。
- Runner 负责执行任务，并在执行时将两者“撮合”在一起。

区别了之前的`set_default_openai_client(client=client, use_for_tracing=False) `全局变量模型设置
- 为了应用于多Agent
- 应用于A/B 测试

设计上使用了`CUSTOM_MODEL_PROVIDER = CustomModelProvider()`动态提供模型
使用`.py`运行


In [ ]:
# 导入所有需要的类和函数
# 从 agents 库导入核心组件，这是构建和运行智能体的基础
from agents import (
    Agent,                      # 用于定义一个智能体（角色、指令、模型等）
    Model,                      # 模型类的基类，用于类型提示
    ModelProvider,              # “模型提供者”的基类，我们要继承它来实现自定义
    OpenAIChatCompletionsModel, # 一个具体的模型实现，用于包装任何兼容OpenAI API的聊天模型
    RunConfig,                  # “运行配置”类，用于在运行时传递特定配置
    Runner,                     # “运行器”，负责执行Agent的任务
    set_tracing_disabled,       # 一个辅助函数，用于关闭详细的日志追踪
)
# 从 openai 库导入异步客户端，用于和模型API通信
from openai import AsyncOpenAI
# 导入 os 和 dotenv 用于加载环境变量（API密钥）
import os
from dotenv import load_dotenv
import asyncio

# --- 第一部分：环境和模型客户端的基础配置 ---

# 加载 .env 文件中的环境变量
load_dotenv()

# 定义我们将要使用的智谱模型名称
chat_model = "glm-4-flash"

# 创建一个专门用于连接“智谱AI”的 AsyncOpenAI 客户端实例
# 这就像是创建了一个专门拨打智谱API热线的“电话机”
client = AsyncOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4/",  # 智谱API的地址
    api_key=os.getenv('zhipu_key'),                   # 从环境变量中读取智谱的API密钥
)

# chat_model = "mistral-small-latest"
# client = AsyncOpenAI(
#     base_url="https://api.mistral.ai/v1",
#     api_key=os.getenv('mistral_key'),
# )

# --- 第二部分：定义核心的“模型提供者” ---

# 我们定义一个自己的类，继承自 ModelProvider
# 它的作用就像一个“模型工厂”，专门生产我们想要的模型实例
class CustomModelProvider(ModelProvider):
    # 必须实现 get_model 方法，这是 ModelProvider 的“合同要求”
    def get_model(self) -> Model:
        # 这个方法被调用时，它会创建并返回一个配置好的模型实例
        # 这里，它返回一个使用上面配置好的智谱客户端和模型名称的模型对象
        return OpenAIChatCompletionsModel(model=chat_model, openai_client=client)

# 创建这个“模型工厂”的一个全局实例，方便后面使用
CUSTOM_MODEL_PROVIDER = CustomModelProvider()

# --- 第三部分：创建 Agent 并配置运行 ---

# 禁用框架的详细日志追踪，让输出更简洁
set_tracing_disabled(disabled=True)

# 创建一个 Agent 实例
agent = Agent(
    name="Assistant",
    # 注意：这里我们仍然为 Agent 指定了一个“默认”模型。
    # 但在接下来的运行中，这个默认模型将被“覆盖”。
    model=OpenAIChatCompletionsModel(model=chat_model, openai_client=client),
    instructions="You are a helpful assistant"
)

# 定义主函数
def main():
    # 使用 Runner.run_sync 来同步执行 Agent 的任务
    result = Runner.run_sync(
        agent,  # 要运行的Agent
        "外观设计专利保护期多少年？",  # 要Agent处理的任务（问题）
        
        # --- 这是最关键的一步 ---
        # 我们创建了一个 RunConfig 对象，并把我们的“模型工厂”传了进去
        # 这就像是给这次任务下达了一个“特殊指令”：不要用默认模型，用这个提供者给的模型！
        run_config=RunConfig(model_provider=CUSTOM_MODEL_PROVIDER),
    )
    # 打印Agent执行任务后的最终输出
    print(result.final_output)

# Python脚本的入口点
if __name__ == "__main__":
    main()

目前，以这种方式配置 智谱、deepseek、和mistral都不支持异步运行，需要用Runner.run_sync 去运行。--2025/10/18

设置模型的配置：温度值、语言等

In [ ]:
# 创建一个名为 "English agent" 的 Agent 实例
english_agent = Agent(
    name="English agent",
    # 设定Agent的系统指令：你只说英语
    instructions="You only speak English",
    
    # --- 关键部分 ---
    # 创建一个ModelSettings实例，并将其传给 model_settings 参数
    model_settings=ModelSettings(temperature=0.1,max_tokens=10,),#max_token限制输出的token
)

## 日志配置：
- 用于追踪Agent“思考过程”中的详细日志信息，帮助开发者调试和理解Agent的行为。
- 清晰回溯Agent的每一步操作，包括收到的用户输入、Agent的思考过程、调用的工具和最终的输出结果。

### 定义 (Definition)：日志的“级别”和“处理器”是什么？
1. 日志记录器 (Logger)：你可以把它想象成一个专门负责记录信息的“书记员”。教程中提到了两个“书记员”：`openai.agents` 和 `openai.agents.tracing`。它们分别负责记录不同方面的日志。
2. 日志级别 (Log Level)：这是给信息打上的“重要性标签”。从低到高通常是：
- `DEBUG`：最详细的信息，用于深入调试，比如Agent的每一次内心思考。
- `INFO`：常规信息，用于追踪程序正常运行的流程，比如“Agent已开始执行任务”。
- `WARNING`：警告信息，表示发生了意料之外的事情，但程序还能继续运行。
- `ERROR`：错误信息，表示发生了严重问题，程序可能无法完成任务。

3. 处理器 (Handler)：这是决定日志信息去向的组件。`StreamHandler`会把信息打印到你的终端（`stdout`或`stderr`）。FileHandler则会把信息写入到一个文件中。

4. 默认情况下，`openai-agents`框架只显示`WARNING`及以上级别的信息，这就是为什么你平时运行可能看不到太多中间过程的原因。

### 方法一：快速开启“啰嗦模式” (enable_verbose_stdout_logging)

In [ ]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging()

作用：这个函数是一个“一键开关”。它会找到`openai.agents`和`openai.agents.tracing`这两个日志记录器，将它们的日志级别都设置为`DEBUG`（最低级别，意味着所有信息都显示），并添加一个`StreamHandler`，让所有信息都打印到你的终端上。

何时使用：当你刚开始开发，想要看到Agent所有内部思考和决策的完整流程时，用它就对了。

### 方法二：专业的手动精细控制 (使用logging库)

In [ ]:
import logging

# 1. 按名称获取你想要配置的日志记录器
logger = logging.getLogger("openai.agents")  # 也可以是 "openai.agents.tracing"

# 2. 设置你希望看到的最低日志级别
logger.setLevel(logging.DEBUG)  # 显示所有信息
logger.setLevel(logging.INFO)   # 只显示INFO、WARNING、ERROR
logger.setLevel(logging.WARNING) # 只显示WARNING和ERROR

# 3. 添加一个处理器，告诉日志信息应该输出到哪里
# StreamHandler会默认输出到终端的stderr
logger.addHandler(logging.StreamHandler())

- 作用：这种方法让你能够精确控制哪个记录器、显示什么级别的信息、以及把信息发送到哪里。例如，你可以只看`openai.agents`的`INFO`信息，而完全忽略`tracing`的日志。

- 何时使用：当你的应用变得复杂，或者你想将日志保存到文件而不是刷屏终端时，就需要使用这种方法。

## 如何处理日志中的敏感数据
`openai-agents`框架很贴心地提供了通过环境变量来禁用这些敏感日志的功能。
操作方法：在运行你的Python脚本之前，在终端中设置以下环境变量:
写入`.env`文件中.

In [ ]:
# 禁用记录模型的输入和输出
export OPENAI_AGENTS_DONT_LOG_MODEL_DATA=1
# 禁用记录工具的输入和输出
export OPENAI_AGENTS_DONT_LOG_TOOL_DATA=1